# Sentiment Analysis sulle Recensioni di Yelp
La Sentiment Analysis è il processo di identificazione dell'emozione espressa in un testo, positiva o negativa.<br><br>
In questo notebook useremo Spark e la sua MLlib per costruire un modello di Sentiment Analysis usando il dataset messo a disposizione da Yelp, una famossisima applicazione che permette di recensire locali e attività commerciali.

## Procuriamoci il Dataset
Possiamo scaricare il dataset dalla pagina ufficiale, che trovi a [questo indirizzo](https://www.yelp.com/dataset), oppure [tramite Kaggle](https://www.kaggle.com/yelp-dataset/yelp-dataset). Per non uscire dal notebook scarichiamo il dataset usando le API di Kaggle.

In [0]:
!kaggle datasets download yelp-dataset/yelp-dataset

Il dataset si trova all'interno di un'archivio zip, estraiamolo con unzip.

In [0]:
!unzip -o yelp-dataset.zip

L'archivio contiene 4 file json differenti:
* yelp_academic_dataset_business.json
* yelp_academic_dataset_checkin.json
* yelp_academic_dataset_review.json
* yelp_academic_dataset_tip.json
* yelp_academic_dataset_user.json

Ognuno contiene delle informazioni differenti, quello con le recensioni è *yelp_academic_dataset_review.json* che è pesa oltre 5 GB.

## Inizializziamo Spark
Inizializziamo Spark usando i contesti. Perchè no una sessione come abbiamo fatto in precedenza ? Perché dobbiamo specificare la configurazione manualmente, nello specifico dobbiamo modificare la dimensione della memoria del driver.

In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("basic").setMaster("local").set('spark.driver.memory','5g')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

## Importiamo il dataset in un DataFrame
Importiamo il dataset in un DataFrame, trattandosi di un file json possiamo utilizzare la funzione .json.

In [0]:
yelp_df = sqlContext.read.json('yelp_academic_dataset_review.json')
#yelp_df.show(5)

**ATTENZIONE**: Se dovessi ottenere un'errore di tipo *permission denied*, modifica i permessi sul file json e riprova.

In [0]:
!sudo chmod 777 yelp_academic_dataset_review.json

Vediamo quali sono le colonne del DataFrame

In [0]:
yelp_df.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

Abbiamo ben 9 colonne, che sono:
* user_id: identificativo del recensore
* business_id: identificato del business recensito
* review_id: identificativo della recensione
* text: testo della recensione
* date: data della recensione
* stars: valutazione dell'attività (da 1.0 a 5.0).
* useful: numero di utenti che hanno segnato la recensione come uile
* cool: numero degli utenti che hanno segnato la recensione come toga (si dice ancora toga? Forse no).
* funny: numero di utenti che hanno segnato la recensione come divertente.

Le uniche informazioni che a noi interessano sono il testo e la valutazione, creiamo un nuovo DataFrame con soltanto queste colonne.

In [0]:
reviews_df = yelp_df.select(["text", "stars"])
#reviews_df.show(5)

Quante recensioni abbiamo a disposizione ? Non lo so, contiamole.

In [0]:
reviews_df.count()

6685900

Le recensioni sono ben 6.685.900, davvero tante ! Facciamo una cosa, cominciamo creando un modello utilizzando soltanto il 1% del dataset.

In [0]:
subreviews_df = reviews_df.sample(False, 0.01, seed=0)
subreviews_df.count()

67136

## Preprocessing del testo
Ora dobbiamo processare il testo delle recensioni per renderlo un buon input per una modello di machine learning. Iniziamo rimuovendo tutta la punteggiatura da ogni frase. Definiamo una funzione per farlo.

In [0]:
import string

def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))

remove_punct("...che cacchio dici !!!1!")

'che cacchio dici 1'

Utilizziamo la funzione *udf* (User Defined Function - Funzione Definita dall'Utente) per creare una funzione spark partendo da quella che abbiamo definito noi per la rimozione della punteggiatura.

In [0]:
from pyspark.sql.functions import udf

punct_remove = udf(lambda s: remove_punct(s))

Fatto questo applichiamo la funzione alla colonna text, per rimuovere la punteggiatura da ogni recensione.

In [0]:
subreviews_df = subreviews_df.withColumn("text", punct_remove(reviews_df["text"]))
#reviews_df.show(5)

Fantastico ! Prossimo step, eseguire la **Tokenizzazione**, che ci serve per estrarre i **Token** dal testo, cioè le sue parti costituenti (le parole insomma). Per farlo possiamo usare la classe *Tokenizer* di MLlib.

In [0]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_df = tokenizer.transform(subreviews_df)

#words_df.show(5)

Ora abbiamo ogni recensione rappresentata da una lista di parole, molte di queste parole sono superflue e non portano informazioni utili ai fini della sentiment analysis. Tali parole sono dette StopWords ed è buona pratica rimuoverle, possiamo farlo utilizzando la classe *StopWordsRemover* di MLlib.

In [0]:
from pyspark.ml.feature import StopWordsRemover

stopwords = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = stopwords.transform(words_df)

#words_df.show(5)

Adesso abbiamo ogni recensione rappresentata da una lista di parole utili, ma un modello di Machine Learning non.

In [0]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol='filtered', outputCol='features')
cv_model = cv.fit(words_df)
cv_df = cv_model.transform(words_df)

Ora scartiamo pure tutte le colonne intermedie che abbiamo creato tenendo soltanto quelle che ci serviranno per realizzare il modello, features e stars.

In [0]:
cv_df = cv_df.select(["features","stars"])
#data_df.show(5)

## Quali sono le recensioni negative ?
Come abbiamo già detto le recensioni sono accompagnate da una valutazione che va da 1.0 a 5.0 stelle, etichettiamo come positive le recensioni con una valutazione di almeno 3.5 stelle, mentre come negative quelle con una valutazione inferiore alle 2.5 stelle. <br>
Le recensioni con 3 stelle sono tendenzialmente neutre, quindi rimuoviamole dal dataframe.

In [0]:
from pyspark.sql.functions import when

cv_df = cv_df.filter("stars != 3.0")
cv_df = cv_df.withColumn("label", when(cv_df["stars"]>=3.5, 1).otherwise(0))

Utilizziamo il metodo *randomSplit* per dividere il DataFrame in due:
* un DataFrame per l'addestramento del modello che conterrà il 70% degli esempi.
* un DataFrame per il testing del modello che conterrà il restante 30% degli esempi.

In [0]:
train_df, test_df = cv_df.randomSplit([0.7, 0.3])

Ora possiamo creare il modello, utilizziamo una Regressione Logistica.

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_df)

Valutiamo il modello addestrato sul DataFrame di test.

In [0]:
evaluation = model.evaluate(test_df)
print(evaluation.accuracy)
print(evaluation.precisionByLabel)
print(evaluation.recallByLabel)

0.9167448793684094
[0.8501123595505617, 0.9384469003879089]
[0.8181228373702422, 0.9505523018756024]


## Creiamo un modello con tutti i dati

Bene, adesso addestriamo il modello utilizzando il dataset per intero con tutti le sue 6.685.900 recensioni.
<br><br>
**ATTENZIONE** Il processo di creazione del modello richiede molte risorse di calcolo e, di conseguenza, tempo. Dovresti utilizzare almeno una macchina EC2 di tipo t3a.large (costo ~7 centesimi l'ora) o meglio ancora un cluster con EMR.
<br><br>
Rimuoviamo la punteggiatura dal testo.

In [0]:
data_df = reviews_df.withColumn("text", punct_remove(reviews_df["text"]))

Eseguiamo la tokenizzazione e rimuoviamo le stop words.

In [0]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
data_df = tokenizer.transform(data_df)

stopwords = StopWordsRemover(inputCol="words", outputCol="filtered")
data_df = stopwords.transform(data_df)

Questa volta, piuttosto che usare un semplice modello Bag of Words per la rappresentazione delle parole, usiamo un modello più sofisticato, cioè il **TF-IDF** (Term Frequency - Inverse Document Frequency) che assegna un peso maggiore alle parole più rare e penalizza quelle più comuni.

In [0]:
from pyspark.ml.feature import HashingTF, IDF

hashing_tf = HashingTF(inputCol='filtered', outputCol='raw_features')
data_df = hashing_tf.transform(data_df)

idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(data_df)
data_df = idf_model.transform(data_df)

Creiamo la colonna per il target.

In [0]:
data_df = data_df.filter("stars != 3.0")
data_df = data_df.withColumn("label", when(data_df["stars"]>=3.5, 1).otherwise(0))

Dividiamo il dataframe in set di addestramento e di test, avendo moltissimi esempi possiamo anche ridurre la dimensione del set di test all'1%.

In [0]:
train_df, test_df = data_df.randomSplit([0.9, 0.1])

Creiamo il modello e addestriamolo.

In [0]:
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_df)

Valutiamolo sul set di test.

In [0]:
evaluation = model.evaluate(test_df)
print(evaluation.accuracy)
print(evaluation.precisionByLabel)
print(evaluation.recallByLabel)

0.9477827461000538


KeyboardInterrupt: 

## Testiamo il Modello

In [0]:
reviews = [
        ("World's Largest Entertainment McDonald's","Lazy staff who do not want to serve u would rather stand in corners in groups talking stood at counter 10 minutes with all staff refusing eye contact as fear of having to serve u supervisor went over and shouted at staff they all stood there shrugging shoulders not wanting t serve u then when orders were ready staff came with trays dragging feet and rolling eyes then it was cold horrible won’t return !!"),
        ("Disnayland Paris","Went here 2x with my husband and found it more magical the 2nd time. Still the happiest place on earth on my list. It gets better and better."),
        ("58 Tour Eiffel Restaurant","What an experience! what a VIEW!. what a meal!!... Delicious, fine dining. excellent0 excellent service and food. A memory of a lifetime"),
        ("Ristorante Cracco","If you want to start your trip in Milan with good mood, for sure you have to avoid this restaurant - the worst pizza we had and the smallest portion of pasta! And incompatible price for that everything! Even, I am really angry, because this is not my first visit in Italy and not first pizza and I feel myself like ....!!!!"),
        ("Happy Wok","Stay away as far as you can, unless you like goopy tables and mass produced food that appeared to be sitting out for too long. It wasn’t a nice experience and we will not attempt to go back under any circumstance"),
        ("Pepe in grani","45 minutes driving from Naples center. Worth every moment on the way. The best and the most unique pizza I ever tasted. Very nice place, every centimeter was well though and planned before implemented. Nice terrace on top for those like the view. Very welcoming crew, great and fast service. Recommend to order the tasting option for those coming in parties of four. ")
    ]

test_df = sqlContext.createDataFrame(reviews, ["location","text"] )

In [0]:
test_df = test_df.withColumn("text", punct_remove(test_df["text"]))

In [0]:
test_df = tokenizer.transform(test_df)
test_df = stopwords.transform(test_df)

In [0]:
test_df = hashing_tf.transform(test_df)
test_df = idf_model.transform(test_df)

In [0]:
pred_df = model.transform(test_df)

In [0]:
#pred_df = pred_df.select(["text", "label"])
pred_df.show()